[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/cybersecurity/01_Real_Time_Anomaly_Detection.ipynb)

# Real-Time Anomaly Detection - Stream Processing & Temporal KGs

## Overview

This notebook demonstrates **real-time anomaly detection** using Semantica with focus on **stream ingestion**, **temporal knowledge graphs**, and **pattern detection**. The pipeline streams security logs in real-time, builds temporal knowledge graphs, and detects anomalies using pattern detection.

### Key Features

- **Stream Processing**: Emphasizes real-time log streaming and processing
- **Temporal Knowledge Graphs**: Builds temporal KGs to track events over time
- **Pattern Detection**: Uses graph patterns to identify anomalies
- **Automated Alerting**: Generates alerts for detected anomalies
- **Comprehensive Data Sources**: Multiple security RSS feeds, APIs, and databases
- **Modular Architecture**: Direct use of Semantica modules without core orchestrator

### Learning Objectives

- Ingest security data from multiple sources (RSS feeds, APIs, streams)
- Extract security entities (Logs, Events, IPs, Users, Alerts, Attacks)
- Build temporal security knowledge graphs
- Perform temporal queries and pattern detection
- Detect anomalies using graph reasoning
- Store and query security data using vector stores and graph stores

### Pipeline Flow

```mermaid
graph TD
    A[Data Ingestion] --> B[Document Parsing]
    B --> C[Text Processing]
    C --> D[Entity Extraction]
    D --> E[Relationship Extraction]
    E --> F[Deduplication]
    F --> G[Conflict Detection]
    G --> H[Temporal Knowledge Graph]
    H --> I[Embeddings]
    I --> J[Vector Store]
    H --> K[Temporal Queries]
    K --> L[Pattern Detection]
    L --> M[Reasoning & Anomaly]
    J --> N[GraphRAG Queries]
    M --> N
    H --> O[Graph Store]
    N --> P[Visualization]
    O --> P
    P --> Q[Export]
```

### Data Sources

#### Security RSS Feeds
- **CVE Database**: https://cve.mitre.org/data/downloads/allitems.xml
- **US-CERT Alerts**: https://www.us-cert.gov/ncas/alerts.xml
- **SANS Internet Storm Center**: https://isc.sans.edu/rssfeed.xml
- **Krebs on Security**: https://krebsonsecurity.com/feed/
- **ThreatPost**: https://threatpost.com/feed/
- **BleepingComputer**: https://www.bleepingcomputer.com/feed/

#### Security APIs
- **CVE API**: https://cve.mitre.org/api/
- **MITRE ATT&CK**: https://attack.mitre.org/
- **VirusTotal API**: https://www.virustotal.com/gui/join-us
- **AbuseIPDB API**: https://www.abuseipdb.com/api
- **Shodan API**: https://www.shodan.io/
- **AlienVault OTX**: https://otx.alienvault.com/api

#### Stream Sources
- **Kafka Streams**: Real-time log streaming
- **WebSocket Feeds**: Real-time security event feeds
- **Syslog**: System log streams
- **ELK Stack**: Elasticsearch, Logstash, Kibana

#### Database Links
- **CVE Database**: https://cve.mitre.org/
- **MITRE ATT&CK**: https://attack.mitre.org/
- **NVD (National Vulnerability Database)**: https://nvd.nist.gov/
- **Exploit-DB**: https://www.exploit-db.com/
- **CAPEC (Common Attack Pattern)**: https://capec.mitre.org/

---

## Installation


In [ ]:
%pip install -qU semantica networkx matplotlib plotly pandas faiss-cpu beautifulsoup4 groq sentence-transformers scikit-learn


## Configuration & Setup


In [ ]:
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "your-key-here")

# Configuration constants
EMBEDDING_DIMENSION = 384
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
TEMPORAL_GRANULARITY = "minute"


## Ingesting Security Data from Multiple Sources


In [ ]:
from semantica.ingest import FeedIngestor, StreamIngestor, FileIngestor
import os
from contextlib import redirect_stderr
from io import StringIO

os.makedirs("data", exist_ok=True)

feed_sources = [
    # Security RSS Feeds
    ("US-CERT Alerts", "https://www.us-cert.gov/ncas/alerts.xml"),
    ("SANS ISC", "https://isc.sans.edu/rssfeed.xml"),
    ("Krebs on Security", "https://krebsonsecurity.com/feed/"),
    ("ThreatPost", "https://threatpost.com/feed/"),
    ("BleepingComputer", "https://www.bleepingcomputer.com/feed/"),
]

feed_ingestor = FeedIngestor()
all_documents = []

for feed_name, feed_url in feed_sources:
    try:
        with redirect_stderr(StringIO()):
            feed_data = feed_ingestor.ingest_feed(feed_url, validate=False)
        for item in feed_data.items:
            if not item.content:
                item.content = item.description or item.title or ""
            if item.content:
                if not hasattr(item, 'metadata'):
                    item.metadata = {}
                item.metadata['source'] = feed_name
                all_documents.append(item)
    except Exception:
        continue

# Simulate stream ingestion (in production, use actual Kafka/WebSocket)
if not all_documents:
    security_logs = """
    2024-01-01 10:00:00 - Login attempt from IP 192.168.1.100 user admin
    2024-01-01 10:01:00 - Failed login from IP 192.168.1.100 user admin
    2024-01-01 10:02:00 - Multiple failed logins from IP 192.168.1.100 user admin
    2024-01-01 10:03:00 - Unusual activity detected from IP 192.168.1.100
    2024-01-01 10:04:00 - Alert: Potential brute force attack from IP 192.168.1.100
    2024-01-01 10:05:00 - Login attempt from IP 192.168.1.101 user test
    2024-01-01 10:06:00 - Suspicious file access from IP 192.168.1.102
    2024-01-01 10:07:00 - Multiple connection attempts from IP 192.168.1.103
    """
    with open("data/security_logs.txt", "w") as f:
        f.write(security_logs)
    file_ingestor = FileIngestor()
    all_documents = file_ingestor.ingest("data/security_logs.txt")

documents = all_documents
print(f"Ingested {len(documents)} documents")


In [ ]:
from semantica.parse import DocumentParser

parser = DocumentParser()

parsed_documents = []
for doc in documents:
    try:
        parsed = parser.parse(
            doc.content if hasattr(doc, 'content') else str(doc),
            content_type="text"
        )
        parsed_documents.append(parsed)
    except Exception:
        parsed_documents.append(doc)

documents = parsed_documents


## Normalizing and Chunking Security Logs


In [ ]:
from semantica.normalize import TextNormalizer
from semantica.split import TextSplitter

normalizer = TextNormalizer()
# Use sentence chunking for log line boundaries (structured logs)
splitter = TextSplitter(method="sentence", chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

normalized_documents = []
for doc in documents:
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True,
        lowercase=False
    )
    normalized_documents.append(normalized_text)

chunked_documents = []
for doc_text in normalized_documents:
    try:
        with redirect_stderr(StringIO()):
            chunks = splitter.split(doc_text)
        chunked_documents.extend(chunks)
    except Exception:
        simple_splitter = TextSplitter(method="recursive", chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        chunks = simple_splitter.split(doc_text)
        chunked_documents.extend(chunks)


## Extracting Security Entities


In [ ]:
from semantica.semantic_extract import NERExtractor

entity_extractor = NERExtractor(
    method="llm",
    provider="groq",
    llm_model="llama-3.1-8b-instant",
    temperature=0.0
)

all_entities = []
for chunk in chunked_documents:
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        entities = entity_extractor.extract_entities(
            chunk_text,
            entity_types=["Log", "Event", "IP", "User", "Alert", "Attack"]
        )
        all_entities.extend(entities)
    except Exception:
        continue

ips = [e for e in all_entities if e.label == "IP" or "ip" in e.label.lower()]
users = [e for e in all_entities if e.label == "User" or "user" in e.label.lower()]
alerts = [e for e in all_entities if e.label == "Alert" or "alert" in e.label.lower()]

print(f"Extracted {len(ips)} IPs, {len(users)} users, {len(alerts)} alerts")


## Extracting Security Relationships


In [ ]:
from semantica.semantic_extract import RelationExtractor

relation_extractor = RelationExtractor(
    method="llm",
    provider="groq",
    llm_model="llama-3.1-8b-instant",
    temperature=0.0
)

all_relationships = []
for chunk in chunked_documents:
    chunk_text = chunk.text if hasattr(chunk, 'text') else str(chunk)
    try:
        relationships = relation_extractor.extract_relations(
            chunk_text,
            entities=all_entities,
            relation_types=["from", "attempts", "triggers", "detects", "associated_with", "causes"]
        )
        all_relationships.extend(relationships)
    except Exception:
        continue

print(f"Extracted {len(all_relationships)} relationships")


## Resolving Duplicate Events


In [ ]:
from semantica.deduplication import DuplicateDetector

duplicate_detector = DuplicateDetector(
    similarity_threshold=0.85,
    method="semantic"
)

deduplicated_entities = duplicate_detector.detect_duplicates(all_entities)
merged_entities = duplicate_detector.merge_duplicates(deduplicated_entities)

print(f"Deduplicated {len(all_entities)} entities to {len(merged_entities)} unique entities")


## Detecting Security Conflicts


In [ ]:
from semantica.conflicts import ConflictDetector

conflict_detector = ConflictDetector()

conflicts = conflict_detector.detect_conflicts(merged_entities, all_relationships)

if conflicts:
    resolved = conflict_detector.resolve_conflicts(conflicts, strategy="highest_confidence")
    print(f"Detected {len(conflicts)} conflicts, resolved {len(resolved)}")
else:
    print("No conflicts detected")


## Building Temporal Security Knowledge Graph


In [ ]:
from semantica.kg import GraphBuilder

graph_builder = GraphBuilder(
    merge_entities=True,
    resolve_conflicts=True,
    entity_resolution_strategy="fuzzy",
    enable_temporal=True,
    temporal_granularity=TEMPORAL_GRANULARITY
)

kg_sources = [{
    "entities": [{"text": e.text, "type": e.label, "confidence": e.confidence} for e in merged_entities],
    "relationships": [{"source": r.source, "target": r.target, "type": r.label, "confidence": r.confidence} for r in all_relationships]
}]

kg = graph_builder.build(kg_sources)

entities_count = len(kg.get('entities', []))
relationships_count = len(kg.get('relationships', []))
print(f"Graph: {entities_count} entities, {relationships_count} relationships")


## Generating Embeddings for Events and IPs


In [ ]:
from semantica.embeddings import EmbeddingGenerator

embedding_gen = EmbeddingGenerator(
    provider="sentence_transformers",
    model=EMBEDDING_MODEL
)

event_texts = [f"{e.text} {getattr(e, 'description', '')}" for e in all_entities if e.label in ["Event", "Log", "Alert"]]
event_embeddings = embedding_gen.generate_embeddings(event_texts)

ip_texts = [f"{ip.text} {getattr(ip, 'description', '')}" for ip in ips]
ip_embeddings = embedding_gen.generate_embeddings(ip_texts)

print(f"Generated {len(event_embeddings)} event embeddings and {len(ip_embeddings)} IP embeddings")


## Populating Vector Store


In [ ]:
from semantica.vector_store import VectorStore

vector_store = VectorStore(backend="faiss", dimension=EMBEDDING_DIMENSION)

event_ids = vector_store.store_vectors(
    vectors=event_embeddings,
    metadata=[{"type": "event", "name": e.text, "label": e.label} for e in all_entities if e.label in ["Event", "Log", "Alert"]]
)

ip_ids = vector_store.store_vectors(
    vectors=ip_embeddings,
    metadata=[{"type": "ip", "name": ip.text, "label": ip.label} for ip in ips]
)

print(f"Stored {len(event_ids)} event vectors and {len(ip_ids)} IP vectors")


## Temporal Graph Queries


In [ ]:
from semantica.kg import TemporalGraphQuery

temporal_query = TemporalGraphQuery(
    enable_temporal_reasoning=True,
    temporal_granularity=TEMPORAL_GRANULARITY
)

query_results = temporal_query.query_at_time(
    kg,
    query={"type": "Alert"},
    at_time="2024-01-01 10:04:00"
)

evolution = temporal_query.analyze_evolution(kg)
temporal_patterns = temporal_query.detect_temporal_patterns(kg, pattern_type="sequence")

print(f"Temporal queries: {len(query_results)} alerts at query time")
print(f"Temporal patterns detected: {len(temporal_patterns)}")


## Detecting Anomaly Patterns


In [ ]:
from semantica.kg import GraphAnalyzer

graph_analyzer = GraphAnalyzer()

# Detect suspicious IPs
suspicious_ips = []
for entity in kg.get("entities", []):
    if entity.get("type") == "IP":
        related_rels = [r for r in kg.get("relationships", []) 
                        if r.get("source") == entity.get("id") or r.get("target") == entity.get("id")]
        if any("alert" in str(r.get("type", "")).lower() or "attack" in str(r.get("type", "")).lower() 
               for r in related_rels):
            suspicious_ips.append(entity)

# Detect anomaly patterns (multiple failed logins, unusual activity)
anomaly_patterns = []
for ip in ips[:10]:
    ip_name = ip.text
    paths = graph_analyzer.find_paths(
        kg,
        source=ip_name,
        target_type="Alert",
        max_hops=2
    )
    if len(paths) > 0:
        anomaly_patterns.append({
            'ip': ip_name,
            'alert_count': len(paths),
            'pattern': 'suspicious_activity'
        })

print(f"Pattern detection: {len(anomaly_patterns)} anomaly patterns found")
print(f"Suspicious IPs: {len(suspicious_ips)}")


## Reasoning and Anomaly Detection


In [ ]:
from semantica.reasoning import Reasoner

reasoner = Reasoner()

reasoner.add_rule("IF IP attempts Event AND Event type failed_login AND Event count > 3 THEN IP triggers Alert")
reasoner.add_rule("IF User from IP AND IP triggers Alert THEN User associated_with Alert")

inferred_facts = reasoner.infer_facts(kg)

anomaly_paths = reasoner.find_paths(
    kg,
    source_type="IP",
    target_type="Alert",
    max_hops=2
)

print(f"Inferred {len(inferred_facts)} facts")
print(f"Found {len(anomaly_paths)} anomaly paths")


## Storing Security Knowledge Graph (Optional)


In [ ]:
from semantica.graph_store import GraphStore

# Optional: Store to persistent graph database
# graph_store = GraphStore(backend="neo4j", uri="bolt://localhost:7687", user="neo4j", password="password")
# graph_store.store_graph(kg)

print("Graph store configured (commented out for demo)")


## GraphRAG: Hybrid Vector + Graph Queries


In [ ]:
from semantica.context import AgentContext

context = AgentContext(vector_store=vector_store, knowledge_graph=kg)

query = "What IPs are associated with security alerts?"
results = context.retrieve(
    query,
    max_results=10,
    use_graph=True,
    expand_graph=True,
    include_entities=True,
    include_relationships=True
)

print(f"GraphRAG query: '{query}'")
print(f"\nRetrieved {len(results)} results:\n")
for i, result in enumerate(results[:5], 1):
    print(f"{i}. Score: {result.get('score', 0):.3f}")
    print(f"   Content: {result.get('content', '')[:200]}...")
    if result.get('related_entities'):
        print(f"   Related entities: {len(result['related_entities'])}")
    print()


## Visualizing the Temporal Security Knowledge Graph


In [ ]:
from semantica.visualization import KGVisualizer

visualizer = KGVisualizer()
visualizer.visualize(
    kg,
    output_path="anomaly_detection_kg.html",
    layout="temporal",
    node_size=20
)

print("Visualization saved to anomaly_detection_kg.html")


## Exporting Results


In [ ]:
from semantica.export import GraphExporter

exporter = GraphExporter()
exporter.export(kg, output_path="anomaly_detection_kg.json", format="json")
exporter.export(kg, output_path="anomaly_detection_kg.graphml", format="graphml")
exporter.export(kg, output_path="anomaly_detection_alerts.csv", format="csv")

print("Exported knowledge graph to JSON, GraphML, and CSV formats")
